# **PyPSA Demystified**
_Core concepts, best practices and new features_

<div style="display: flex; align-items: center; justify-content: space-between; gap: 1rem; flex-wrap: wrap;">

  <div style="flex: 1; min-width: 250px;">
    <p><strong>Author:</strong><br>
    Priyesh Gosai – Energy Systems Modeller<br>
    Innovate for Impact <br>
    priyesh@innovateimpact.com </p>

Access the accompanying presentation at this [link](https://docs.google.com/presentation/d/1pcAsqgGERAGmIWMsiKo3ljEMb4HT5iKa/edit?slide=id.p15#slide=id.p15).


In [ ]:
#@title Connect to Google Drive {display-mode:"form"}
CONNECT_TO_DRIVE = True #@param {type:"boolean"}

from google.colab import drive
import os

if CONNECT_TO_DRIVE:
    # Mount Google Drive
    drive.mount('/content/drive')

    # Define the desired working directory path
    working_dir = '/content/drive/MyDrive/hello-pypsa'

    # Create the directory if it doesn't exist
    if not os.path.exists(working_dir):
        os.makedirs(working_dir)
        print(f"Directory '{working_dir}' created.")
    else:
        print(f"Directory '{working_dir}' already exists.")

    # Change the current working directory
    os.chdir(working_dir)

    print(f"Current working directory: {os.getcwd()}")
else:
    print("Not connecting to Google Drive.")

# Install packages for Google Colab



In [ ]:
!pip install pypsa pypsa[excel] folium mapclassify
!pip install git+https://github.com/PriyeshGosai/pypsa_network_viewer.git

# **Example 1:**

Programatically builds a one bus network with a generator and fixed load. There are 8760 snapshots that represent 2025.



Create the network object The network object stores all static and timeseries data using pandas dataframes.

In [ ]:
import pypsa
import pandas as pd

n = pypsa.Network()

Programatically add components

In [ ]:
n.add('Bus','Location A')
n.add('Load','Load A',bus = 'Location A',p_set = 100)
n.add('Generator','Generator A',bus = 'Location A', p_nom = 200, marginal_cost = 1)
n.set_snapshots(pd.date_range('2025-01-01', '2025-12-31 23:00', freq='h'))

Observe components stored as `Pandas dataframes`

Component Kahoot

In [ ]:
n.generators

In [ ]:
n.loads

In [ ]:
n.buses

Optional Step to create the model without solving.

This uses `linopy` to build all the constraints.

View all the constraints and variables.

In [ ]:
n.optimize.create_model()

In [ ]:
n.optimize()

In [ ]:
n.generators_t.p.plot()

In [ ]:
n.loads_t.p.plot()

In [ ]:
n.buses_t.marginal_price.plot()

In [ ]:
from pypsa_network_viewer import html_viewer
html_file_basic = html_viewer(
    n,
    file_name='basic_network.html',
    title='Basic Network Model'
)

In [ ]:
basic_network_file = 'basic_network.xlsx'
n.export_to_excel(basic_network_file)

# **Example 2:**

Use an existing network to view more complex features

In [ ]:
import pypsa

n = pypsa.examples.ac_dc_meshed()

View the network on a map

In [ ]:
n.plot.explore()

View all the components in the network

In [ ]:
n.buses

In [ ]:
n.generators

Constraints applied to generators based on constraints at time steps.

e.g. Maximum solar/wind or for conventional generators that have restrictions due to cooling systems.

In [ ]:
n.generators_t.p_max_pu

In [ ]:
n.lines

In [ ]:
n.links

In [ ]:
n.loads

Timeseries data for loads

In [ ]:
n.loads_t.p_set

In [ ]:
n.loads_t.p_set.plot()

Solve the model

In [ ]:
n.optimize()

View all the constraints

In [ ]:
n.model

View Results

In [ ]:
n.generators_t.p.plot()

In [ ]:
n.links_t.p0.plot()

In [ ]:
n.lines_t.p0.plot()

In [ ]:
from pypsa_network_viewer import html_viewer
html_file_acdc = html_viewer(
    n,
    file_name='ac_dc_network.html',
    title='AC-DC Meshed Example Network Model'
)

In [ ]:
solved_ac_dc = 'solved_ac-dc.xlsx'


n.export_to_excel(solved_ac_dc)




# **Example 3:**

Import a network from an excel file

Download the file from Google Drive (view folder structure using the folder icon on the left)

In [ ]:
!wget "https://docs.google.com/spreadsheets/d/1s_3NPMUlv8XU0PwKFw1it7rdvuY8D--g/export?format=xlsx&id=1s_3NPMUlv8XU0PwKFw1it7rdvuY8D--g" -O network_ZA.xlsx

In [ ]:
import pypsa
import pandas as pd
pd.set_option('plotting.backend', 'plotly')


n_ZA = pypsa.Network('network_ZA.xlsx')

# Reduce the number of timesteps
days = 30*24
n_ZA.snapshots = n_ZA.snapshots[:days]

In [ ]:
n.buses

In [ ]:
n_ZA.generators

Constraints applied to generators based on constraints at time steps.

e.g. Maximum solar/wind or for conventional generators that have restrictions due to cooling systems.

In [ ]:
n_ZA.generators_t.p_max_pu

In [ ]:
n_ZA.lines

In [ ]:
n_ZA.links

In [ ]:
n_ZA.loads

In [ ]:
n_ZA.storage_units

Timeseries data for loads

In [ ]:
n_ZA.loads_t.p_set

In [ ]:
n_ZA.loads_t.p_set.plot()

Solve the model

In [ ]:
n_ZA.optimize()

View all the constraints

In [ ]:
n_ZA.model

View Results

In [ ]:
n_ZA.generators_t.p.plot()

In [ ]:
n_ZA.links_t.p0.plot()

In [ ]:
from pypsa_network_viewer import html_viewer
html_file = html_viewer(
    n_ZA,
    file_name='ZA_network.html',
    title='South African Network Model'
)

In [ ]:
html_file

In [ ]:
excel_solved = 'solved_ZA.xlsx'
n_ZA.export_to_excel(excel_solved)

The next line will download the solved models in excel files and as a html files containing all the information from the network.

In [ ]:
from google.colab import files
import zipfile

# Create single zip with all files
with zipfile.ZipFile('modelling_results.zip', 'w') as zipf:
    zipf.write(html_file)
    zipf.write(excel_solved)
    zipf.write(solved_ac_dc)
    zipf.write(html_file_acdc)
    zipf.write(basic_network_file)
    zipf.write(html_file_basic)

# Download the zip
files.download('modelling_results.zip')

# **Exercise Code**



In [ ]:
import pypsa
import pandas as pd
pd.set_option('plotting.backend', 'plotly')


n_ZA_expansion = pypsa.Network('network_ZA.xlsx')

# Reduce the number of timesteps
days = 30*24
n_ZA_expansion.snapshots = n_ZA_expansion.snapshots[:days]

Solve the model

In [ ]:
n_ZA_expansion.optimize()

View all the constraints

View Results

In [ ]:
n_ZA_expansion.generators_t.p.plot()

In [ ]:
n_ZA_expansion.links_t.p0.plot()

In [ ]:
from pypsa_network_viewer import html_viewer
html_file_expansion = html_viewer(
    n_ZA_expansion,
    file_name='ZA_network_expansion.html',
    title='South African Network Model'
)

In [ ]:
html_file

In [ ]:
excel_solved_expansion = 'solved_ZA_expansion.xlsx'
n_ZA.export_to_excel(excel_solved_expansion)

The next line will download the solved models in excel files and as a html files containing all the information from the network.

In [ ]:
from google.colab import files
import zipfile

# Create single zip with all files
with zipfile.ZipFile('modelling_results_expansion.zip', 'w') as zipf:
    zipf.write(html_file_expansion)
    zipf.write(excel_solved_expansion)

# Download the zip
files.download('modelling_results_expansion.zip')